In [31]:
import numpy as np
import time
import scipy
import torch
from concurrent.futures import ProcessPoolExecutor

from gep_problem import GEPProblem
from gep_config_parser import *
from gep_primal_dual_main_refactored import prep_data



In [32]:
CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"
SAMPLE_DURATION = 24
# SAMPLE_DURATION = 120

## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

Parsing the config file


In [33]:
experiment_instance = experiment["experiments"][0]
data = prep_data(experiment_instance, train=0.002, valid=0.002, test=0.996, sample_duration=SAMPLE_DURATION)

Wrangling the input data
Creating problem instance
Populating ineq constraints
Populating eq constraints
Creating objective coefficients
Creating input for NN: X
Size of train set: 0
Size of val set: 0
Size of test set: 365


In [34]:
# ! My ALM functions:
def obj_fn_ALM(Yi):
    # Objective function evaluated for Y
    # return (
    #     0.5 * Yi.T @ self.Q_np @ Yi + self.p_np.T @ Yi
    # )  # Ensure Q_np: (100, 100), p_np: (100,)

    return data.obj_fn(torch.tensor(Yi).unsqueeze(0)).squeeze().numpy()

def eq_resid_ALM(Yi, eq_cm, eq_rhs):
    # Equality residuals (X - A * Y)
    # return Xi - self.A_np @ Yi  # A_np: (50, 100), X: (50,), Y: (100,)
    Yi = torch.tensor(Yi).unsqueeze(0)
    return data.eq_resid(Yi, eq_cm, eq_rhs).squeeze().numpy()

def ineq_resid_ALM(Yi, ineq_cm, ineq_rhs):
    # Inequality residuals (G * Y - h)
    # return self.G_np @ Yi - self.h_np  # G_np: (50, 100), h_np: (50,)
    Yi = torch.tensor(Yi).unsqueeze(0)
    return data.ineq_resid(Yi, ineq_cm, ineq_rhs).squeeze().numpy()

def L_rho(Yi, eq_cm, ineq_cm, eq_rhs, ineq_rhs, m, l, rho):
    # Compute inequality and equality residuals
    ineq_resid = ineq_resid_ALM(Yi, ineq_cm, ineq_rhs)# Should be (50,)
    eq_resid = eq_resid_ALM(Yi, eq_cm, eq_rhs)  # Should be (50,)

    # Augmented Lagrangian computation
    return (
        obj_fn_ALM(Yi)
        + m.T @ ineq_resid  # Linear term for inequality
        + l.T @ eq_resid  # Linear term for equality
        + (rho / 2)
        * (
            np.linalg.norm(np.maximum(ineq_resid, 0), 2) ** 2
            + np.linalg.norm(eq_resid, 2) ** 2
        )
    )

def solve_instance(i, Xi, eq_cm, ineq_cm, eq_rhs, ineq_rhs, tol=1e-4):
        """
        Solve ALM for a single instance.
        """
        mu_k = np.zeros(data.nineq)  # (50,)
        lamb_k = np.zeros(data.neq)  # (50,)
        rho = 1
        tau = 0.5
        alpha = 10
        K = 20

        epsilon = 1e-4
        rho_max = 5000
        # print(f"sample {i+1} of {833}")
        start_time = time.time()
        # Initial primal point
        Yi = np.random.uniform(-1, 1, data.ydim)  # Shape (100,)

        for k in range(K):
            print(f"{k+1}/{K}")
            # Minimize L_rho for current `mu_k` and `lambda_k`
            res = scipy.optimize.minimize(
                fun=L_rho,
                x0=Yi,
                args=(eq_cm, ineq_cm, eq_rhs, ineq_rhs, mu_k, lamb_k, rho),
                tol=tol,
                method="CG",
                scaling=True,
            )
            y_k = res.x  # Solution for this iteration, shape (100,)

            eq_resid = eq_resid_ALM(y_k, eq_cm, eq_rhs)
            ineq_resid = ineq_resid_ALM(y_k, ineq_cm, ineq_rhs)

            # Update dual variables using y_k
            mu_k = np.maximum(mu_k + rho * ineq_resid, 0)
            lamb_k = lamb_k + rho * eq_resid

            # Calculate v_k to check for convergence
            inf_norm_eq_resid = np.max(np.abs(eq_resid))
            sigma = np.maximum(ineq_resid, -mu_k / rho)
            inf_norm_sigma = np.max(np.abs(sigma))
            v_k = max(inf_norm_eq_resid, inf_norm_sigma)

            if v_k < epsilon:
                print(f"Converged for instance {i+1} at iteration {k+1}")
                break

            if k >= 1 and v_k > (tau * prev_v_k):
                rho = min(alpha * rho, rho_max)
            prev_v_k = v_k
            Yi = y_k  # Use the current solution as the initial guess for the next iteration
            print(f"obj_fn:{obj_fn_ALM(Yi)}")
            print(f"dual obj_fn:{(data.dual_obj_fn(eq_rhs, ineq_rhs, torch.tensor(mu_k.tolist()).unsqueeze(0), torch.tensor(lamb_k.tolist()).unsqueeze(0))).item()}")
            print(f"v_k:{v_k}")

            rhs_eq = eq_rhs
            rhs_ineq = ineq_rhs

            print(f"mu mean:{mu_k.mean()}")

            print(f"lamb mean:{lamb_k.mean()}")

        end_time = time.time()
        print(obj_fn_ALM(Yi))
        return y_k, mu_k, lamb_k, end_time - start_time

def ALM_solve(X, tol=1e-4):
    X_np = X.detach().cpu().numpy() if hasattr(X, "detach") else X

    # Parallel processing
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(solve_instance, range(len(X_np)), X_np))

    # Unpack results
    Y, times = zip(*results)
    total_time = sum(times)
    parallel_time = total_time / len(X_np)
    sols = np.array(Y)

    return sols, total_time, parallel_time

In [35]:
X = data.trainX
eq_cm = data.eq_cm[:1]
ineq_cm = data.ineq_cm[:1]
eq_rhs = data.eq_rhs[:1]
ineq_rhs = data.ineq_rhs[:1]
y, mu, lamb, time_taken = solve_instance(0, X, eq_cm, ineq_cm, eq_rhs, ineq_rhs, tol=100)

1/20


TypeError: minimize() got an unexpected keyword argument 'scaling'

In [ ]:
OPTIMAL_OBJ = 8.18648032e+08 # First sample

print(f"Optimality gap (%): {abs(obj_fn_ALM(y) - OPTIMAL_OBJ) / OPTIMAL_OBJ * 100}")
# print(mu)
# print(lamb)

g = ineq_resid_ALM(y, ineq_cm, ineq_rhs)
h = eq_resid_ALM(y, eq_cm, eq_rhs)

# print(g > 0)
print(g[(g>0)].max())
print(g[(g>0)].mean())
print(np.abs(h).mean())

# print(g)
# print(mu[g < 0])

Optimality gap (%): 20.447602219037215
0.23243173084665614
0.017345356135724313
0.0027242787257553878
